# 🗣️ Sentiment Analysis: Aspect-Based Opinion Mining

Applying topic modelling and sentiment analysis techniques in order to extract text aspects and the sentiment expressed towards such aspects.

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/Aspect_Sentiment_Analysis-main')
os.getcwd()

'/content/drive/MyDrive/Aspect_Sentiment_Analysis-main'

In [3]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 6.5 MB/s eta 0:00:00


In [4]:
!apt install stop_words_list

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package stop_words_list


In [5]:
import json
import pandas as pd
import numpy as np
import re
import sys
import nltk
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import collections
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from stop_words_list import stop_words_list
from wn_affect import wn_affect 
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
# read data

# data = pd.read_excel('')

data.rename(columns={'Review Text1#1':'Review'},inplace=True)
data.head()

,Review ID,Month,Product Line,Star Rating,Review
0,675119,2017 / 01,HUMIDIFICATION-PHARMACY,5,I simply love these humidifiers; the amount of...
1,675120,2017 / 01,HUMIDIFICATION-PHARMACY,1,I got this for my daughter and the first time ...
2,675121,2017 / 01,HUMIDIFICATION-PHARMACY,5,Bonjour tout`l monde.Juste un mot pour dire qu...
3,675122,2017 / 01,HUMIDIFICATION-PHARMACY,5,nice
4,675123,2017 / 01,HUMIDIFICATION-PHARMACY,3,Works but very cheap product. I`m sure it cost...


In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/Aspect_Sentiment_Analysis-main/reviews_humidifier_2021.csv')

# df = pd.read_csv('/content/drive/MyDrive/Aspect_Sentiment_Analysis-main/reviews_humidifier_2022.csv')
# df.head()

In [ ]:

# raw_data = []

# for i in data:
#     tmp = []
#     tmp.append(i.split('""')[1])
#     raw_data.append(tmp)
df = pd.DataFrame(data, columns =['Review'])
display(df)

,Review
0,I simply love these humidifiers; the amount of...
1,I got this for my daughter and the first time ...
2,Bonjour tout`l monde.Juste un mot pour dire qu...
3,nice
4,Works but very cheap product. I`m sure it cost...
...,...
27192,Just like the title says you get your humidifi...
27193,The high setting never works for me I constant...
27194,Purchased this for my sons room to increase th...
27195,"Almost immediately after getting it, the filte..."


In [ ]:
# case text as lowercase, remove punctuation, remove extra whitespace in string and on both sides of string

df['remove_lower_punct'] = df['Review'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()

display(df.head(10))

<ipython-input-158-095571329ae4>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['remove_lower_punct'] = df['Review'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()


,Review,remove_lower_punct
0,I simply love these humidifiers; the amount of...,i simply love these humidifiers the amount of ...
1,I got this for my daughter and the first time ...,i got this for my daughter and the first time ...
2,Bonjour tout`l monde.Juste un mot pour dire qu...,bonjour tout l monde juste un mot pour dire qu...
3,nice,nice
4,Works but very cheap product. I`m sure it cost...,works but very cheap product i m sure it costs...
5,Maybe my unit was defective but it consistentl...,maybe my unit was defective but it consistentl...
6,working very nice. No any issues till now,working very nice no any issues till now
7,This product arrived with broken plastic and t...,this product arrived with broken plastic and t...
8,This is my 3rd one of these and I love them. A...,this is my rd one of these and i love them as ...
9,this review is the worst that i`ve never give ...,this review is the worst that i ve never give ...


In [ ]:
# apply sentiment analysis
analyser = SentimentIntensityAnalyzer()

sentiment_score_list = []
sentiment_label_list = []

for i in df['remove_lower_punct'].values.tolist():
    sentiment_score = analyser.polarity_scores(i)

    if sentiment_score['compound'] >= 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Positive')
    elif sentiment_score['compound'] > -0.05 and sentiment_score['compound'] < 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Neutral')
    elif sentiment_score['compound'] <= -0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Negative')
    
df['sentiment'] = sentiment_label_list
df['sentiment score'] = sentiment_score_list

display(df.head(10))

,Review,remove_lower_punct,sentiment,sentiment score
0,I simply love these humidifiers; the amount of...,i simply love these humidifiers the amount of ...,Positive,0.9805
1,I got this for my daughter and the first time ...,i got this for my daughter and the first time ...,Negative,-0.6261
2,Bonjour tout`l monde.Juste un mot pour dire qu...,bonjour tout l monde juste un mot pour dire qu...,Negative,-0.1280
3,nice,nice,Positive,0.4215
4,Works but very cheap product. I`m sure it cost...,works but very cheap product i m sure it costs...,Positive,0.8720
5,Maybe my unit was defective but it consistentl...,maybe my unit was defective but it consistentl...,Positive,0.8248
6,working very nice. No any issues till now,working very nice no any issues till now,Positive,0.1567
7,This product arrived with broken plastic and t...,this product arrived with broken plastic and t...,Negative,-0.6310
8,This is my 3rd one of these and I love them. A...,this is my rd one of these and i love them as ...,Positive,0.8689
9,this review is the worst that i`ve never give ...,this review is the worst that i ve never give ...,Negative,-0.5296


In [ ]:
# tokenise string
import nltk
nltk.download('punkt')
df['tokenise'] = df.apply(lambda row: nltk.word_tokenize(row[1]), axis=1)

display(df.head(10))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Review,remove_lower_punct,sentiment,sentiment score,tokenise
0,I simply love these humidifiers; the amount of...,i simply love these humidifiers the amount of ...,Positive,0.9805,"[i, simply, love, these, humidifiers, the, amo..."
1,I got this for my daughter and the first time ...,i got this for my daughter and the first time ...,Negative,-0.6261,"[i, got, this, for, my, daughter, and, the, fi..."
2,Bonjour tout`l monde.Juste un mot pour dire qu...,bonjour tout l monde juste un mot pour dire qu...,Negative,-0.1280,"[bonjour, tout, l, monde, juste, un, mot, pour..."
3,nice,nice,Positive,0.4215,[nice]
4,Works but very cheap product. I`m sure it cost...,works but very cheap product i m sure it costs...,Positive,0.8720,"[works, but, very, cheap, product, i, m, sure,..."
5,Maybe my unit was defective but it consistentl...,maybe my unit was defective but it consistentl...,Positive,0.8248,"[maybe, my, unit, was, defective, but, it, con..."
6,working very nice. No any issues till now,working very nice no any issues till now,Positive,0.1567,"[working, very, nice, no, any, issues, till, now]"
7,This product arrived with broken plastic and t...,this product arrived with broken plastic and t...,Negative,-0.6310,"[this, product, arrived, with, broken, plastic..."
8,This is my 3rd one of these and I love them. A...,this is my rd one of these and i love them as ...,Positive,0.8689,"[this, is, my, rd, one, of, these, and, i, lov..."
9,this review is the worst that i`ve never give ...,this review is the worst that i ve never give ...,Negative,-0.5296,"[this, review, is, the, worst, that, i, ve, ne..."


In [ ]:
# initiate stopwords from nltk
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')

# add additional missing terms

stop_words.extend(stop_words_list) 

# remove stopwords

df['remove_stopwords'] = df['tokenise'].apply(lambda x: [item for item in x if item not in stop_words])

display(df.head(10))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Review,remove_lower_punct,sentiment,sentiment score,tokenise,remove_stopwords
0,I simply love these humidifiers; the amount of...,i simply love these humidifiers the amount of ...,Positive,0.9805,"[i, simply, love, these, humidifiers, the, amo...","[simply, humidifiers, amount, moisture, awesom..."
1,I got this for my daughter and the first time ...,i got this for my daughter and the first time ...,Negative,-0.6261,"[i, got, this, for, my, daughter, and, the, fi...","[daughter, time, thought, set, incorrectly, wa..."
2,Bonjour tout`l monde.Juste un mot pour dire qu...,bonjour tout l monde juste un mot pour dire qu...,Negative,-0.1280,"[bonjour, tout, l, monde, juste, un, mot, pour...","[bonjour, tout, monde, juste, mot, pour, dire,..."
3,nice,nice,Positive,0.4215,[nice],[nice]
4,Works but very cheap product. I`m sure it cost...,works but very cheap product i m sure it costs...,Positive,0.8720,"[works, but, very, cheap, product, i, m, sure,...","[works, cheap, product, sure, costs, cents, bo..."
5,Maybe my unit was defective but it consistentl...,maybe my unit was defective but it consistentl...,Positive,0.8248,"[maybe, my, unit, was, defective, but, it, con...","[unit, defective, consistently, loud, dripping..."
6,working very nice. No any issues till now,working very nice no any issues till now,Positive,0.1567,"[working, very, nice, no, any, issues, till, now]","[working, nice, issues, till]"
7,This product arrived with broken plastic and t...,this product arrived with broken plastic and t...,Negative,-0.6310,"[this, product, arrived, with, broken, plastic...","[product, arrived, broken, plastic, water, lea..."
8,This is my 3rd one of these and I love them. A...,this is my rd one of these and i love them as ...,Positive,0.8689,"[this, is, my, rd, one, of, these, and, i, lov...","[long, clean, supposed, things, nose, winter, ..."
9,this review is the worst that i`ve never give ...,this review is the worst that i ve never give ...,Negative,-0.5296,"[this, review, is, the, worst, that, i, ve, ne...","[review, worst, amazon, produces, warmer, gas,..."


In [ ]:
# initiate nltk lemmatiser
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

# lemmatise words

df['lemmatise'] = df['remove_stopwords'].apply(lambda x: [wordnet_lemmatizer.lemmatize(y) for y in x]) 

display(df.head(10))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Review,remove_lower_punct,sentiment,sentiment score,tokenise,remove_stopwords,lemmatise
0,I simply love these humidifiers; the amount of...,i simply love these humidifiers the amount of ...,Positive,0.9805,"[i, simply, love, these, humidifiers, the, amo...","[simply, humidifiers, amount, moisture, awesom...","[simply, humidifiers, amount, moisture, awesom..."
1,I got this for my daughter and the first time ...,i got this for my daughter and the first time ...,Negative,-0.6261,"[i, got, this, for, my, daughter, and, the, fi...","[daughter, time, thought, set, incorrectly, wa...","[daughter, time, thought, set, incorrectly, wa..."
2,Bonjour tout`l monde.Juste un mot pour dire qu...,bonjour tout l monde juste un mot pour dire qu...,Negative,-0.1280,"[bonjour, tout, l, monde, juste, un, mot, pour...","[bonjour, tout, monde, juste, mot, pour, dire,...","[bonjour, tout, monde, juste, mot, pour, dire,..."
3,nice,nice,Positive,0.4215,[nice],[nice],[nice]
4,Works but very cheap product. I`m sure it cost...,works but very cheap product i m sure it costs...,Positive,0.8720,"[works, but, very, cheap, product, i, m, sure,...","[works, cheap, product, sure, costs, cents, bo...","[work, cheap, product, sure, cost, cent, bowl,..."
5,Maybe my unit was defective but it consistentl...,maybe my unit was defective but it consistentl...,Positive,0.8248,"[maybe, my, unit, was, defective, but, it, con...","[unit, defective, consistently, loud, dripping...","[unit, defective, consistently, loud, dripping..."
6,working very nice. No any issues till now,working very nice no any issues till now,Positive,0.1567,"[working, very, nice, no, any, issues, till, now]","[working, nice, issues, till]","[working, nice, issue, till]"
7,This product arrived with broken plastic and t...,this product arrived with broken plastic and t...,Negative,-0.6310,"[this, product, arrived, with, broken, plastic...","[product, arrived, broken, plastic, water, lea...","[product, arrived, broken, plastic, water, lea..."
8,This is my 3rd one of these and I love them. A...,this is my rd one of these and i love them as ...,Positive,0.8689,"[this, is, my, rd, one, of, these, and, i, lov...","[long, clean, supposed, things, nose, winter, ...","[long, clean, supposed, thing, nose, winter, r..."
9,this review is the worst that i`ve never give ...,this review is the worst that i ve never give ...,Negative,-0.5296,"[this, review, is, the, worst, that, i, ve, ne...","[review, worst, amazon, produces, warmer, gas,...","[review, worst, amazon, produce, warmer, gas, ..."


In [ ]:
df.shape

(27197, 7)

In [ ]:
# initialise the count vectorizer

vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (2, 2))
                            
# join the processed data to be vectorised

vectors = []

for index, row in df.iterrows():
    vectors.append(", ".join(row[6]))

vectorised = vectorizer.fit_transform(vectors)

print(vectorised)

  (0, 128092)	1
  (0, 66829)	1
  (0, 4150)	1
  (0, 88070)	1
  (0, 6903)	1
  (0, 133353)	1
  (0, 81263)	1
  (0, 161171)	1
  (0, 93653)	1
  (0, 132984)	1
  (0, 47885)	1
  (0, 133303)	1
  (0, 56859)	1
  (0, 110959)	1
  (0, 40781)	1
  (0, 106932)	1
  (0, 88149)	1
  (0, 66588)	1
  (0, 135360)	1
  (0, 92560)	1
  (0, 78071)	1
  (0, 74451)	1
  (0, 91115)	1
  (0, 57063)	1
  (0, 166458)	1
  :	:
  (27194, 41841)	1
  (27194, 907)	1
  (27194, 161771)	1
  (27194, 67587)	1
  (27194, 97116)	1
  (27194, 132748)	1
  (27194, 49554)	1
  (27194, 89331)	1
  (27194, 155600)	1
  (27194, 91934)	1
  (27194, 2312)	1
  (27194, 110280)	1
  (27194, 22416)	1
  (27194, 136087)	1
  (27194, 120342)	1
  (27194, 108345)	1
  (27194, 77459)	1
  (27194, 2223)	1
  (27194, 16819)	1
  (27195, 50535)	1
  (27195, 88644)	1
  (27195, 131677)	1
  (27196, 3814)	1
  (27196, 76771)	1
  (27196, 31689)	1


In [ ]:
# initisalise LDA Model

lda_model = LatentDirichletAllocation(n_components = 100, # number of topics
                                  random_state = 10,          # random state
                                  evaluate_every = -1,      # compute perplexity every n iters, default: Don't
                                  n_jobs = -1,              # Use all available CPUs
                                 )

lda_output = lda_model.fit_transform(vectorised)

# column names

topic_names = ["Topic" + str(i) for i in range(1, lda_model.n_components + 1)]
# make the pandas dataframe

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns = topic_names)

# get dominant topic for each document

dominant_topic = (np.argmax(df_document_topic.values, axis=1)+1)
df_document_topic['Dominant_topic'] = dominant_topic

# join to original dataframes

df = pd.merge(df, df_document_topic, left_index = True, right_index = True, how = 'outer')
display(df.head(10))

,Review,remove_lower_punct,sentiment,sentiment score,tokenise,remove_stopwords,lemmatise,Topic1,Topic2,Topic3,...,Topic92,Topic93,Topic94,Topic95,Topic96,Topic97,Topic98,Topic99,Topic100,Dominant_topic
0,I simply love these humidifiers; the amount of...,i simply love these humidifiers the amount of ...,Positive,0.9805,"[i, simply, love, these, humidifiers, the, amo...","[simply, humidifiers, amount, moisture, awesom...","[simply, humidifiers, amount, moisture, awesom...",0.96,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1
1,I got this for my daughter and the first time ...,i got this for my daughter and the first time ...,Negative,-0.6261,"[i, got, this, for, my, daughter, and, the, fi...","[daughter, time, thought, set, incorrectly, wa...","[daughter, time, thought, set, incorrectly, wa...",0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,13
2,Bonjour tout`l monde.Juste un mot pour dire qu...,bonjour tout l monde juste un mot pour dire qu...,Negative,-0.1280,"[bonjour, tout, l, monde, juste, un, mot, pour...","[bonjour, tout, monde, juste, mot, pour, dire,...","[bonjour, tout, monde, juste, mot, pour, dire,...",0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24
3,nice,nice,Positive,0.4215,[nice],[nice],[nice],0.01,0.01,0.01,...,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,1
4,Works but very cheap product. I`m sure it cost...,works but very cheap product i m sure it costs...,Positive,0.8720,"[works, but, very, cheap, product, i, m, sure,...","[works, cheap, product, sure, costs, cents, bo...","[work, cheap, product, sure, cost, cent, bowl,...",0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4
5,Maybe my unit was defective but it consistentl...,maybe my unit was defective but it consistentl...,Positive,0.8248,"[maybe, my, unit, was, defective, but, it, con...","[unit, defective, consistently, loud, dripping...","[unit, defective, consistently, loud, dripping...",0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,39
6,working very nice. No any issues till now,working very nice no any issues till now,Positive,0.1567,"[working, very, nice, no, any, issues, till, now]","[working, nice, issues, till]","[working, nice, issue, till]",0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,85
7,This product arrived with broken plastic and t...,this product arrived with broken plastic and t...,Negative,-0.6310,"[this, product, arrived, with, broken, plastic...","[product, arrived, broken, plastic, water, lea...","[product, arrived, broken, plastic, water, lea...",0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,25
8,This is my 3rd one of these and I love them. A...,this is my rd one of these and i love them as ...,Positive,0.8689,"[this, is, my, rd, one, of, these, and, i, lov...","[long, clean, supposed, things, nose, winter, ...","[long, clean, supposed, thing, nose, winter, r...",0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,67
9,this review is the worst that i`ve never give ...,this review is the worst that i ve never give ...,Negative,-0.5296,"[this, review, is, the, worst, that, i, ve, ne...","[review, worst, amazon, produces, warmer, gas,...","[review, worst, amazon, produce, warmer, gas, ...",0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,70


In [ ]:
df.columns

Index(['Review', 'remove_lower_punct', 'sentiment', 'sentiment score',
       'tokenise', 'remove_stopwords', 'lemmatise', 'Topic1', 'Topic2',
       'Topic3',
       ...
       'Topic92', 'Topic93', 'Topic94', 'Topic95', 'Topic96', 'Topic97',
       'Topic98', 'Topic99', 'Topic100', 'Dominant_topic'],
      dtype='object', length=108)

In [ ]:
# df.iloc[0:5,7:40]

In [ ]:
# index names
docnames = ['Doc' + str(i) for i in range(len(df))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names_out()
df_topic_keywords.index = topic_names

df_topic_no = pd.DataFrame(df_topic_keywords.idxmax())
df_scores = pd.DataFrame(df_topic_keywords.max())

tmp = pd.merge(df_topic_no, df_scores, left_index=True, right_index=True)
tmp.columns = ['topic', 'relevance_score']

display(tmp)

,topic,relevance_score
01 owned,Topic54,1.01
10 cooling,Topic86,1.01
100 best,Topic84,1.01
100 recommend,Topic14,1.01
1st grandson,Topic77,1.01
...,...,...
zone unit,Topic45,1.01
zone whisper,Topic91,1.01
zumek teresita,Topic42,1.01
zyrtec day,Topic16,1.01


In [ ]:
all_topics = []

for i in tmp['topic'].unique():    
    tmp_1 = tmp.loc[tmp['topic'] == i].reset_index()
    tmp_1 = tmp_1.sort_values('relevance_score', ascending=False).head(1)

    tmp_1['topic'] = tmp_1['topic'].apply(lambda x: str(int(x[5:]) + 1))
    tmp_1['topic'] = tmp_1['topic'].astype(str)
    
    tmp_2 = []
    tmp_2.append(tmp_1['topic'].unique()[0])
    tmp_2.append(list(tmp_1['index'].unique()))
    all_topics.append(tmp_2)

all_topics = pd.DataFrame(all_topics, columns=['Dominant_topic', 'topic_name'])
display(all_topics)

,Dominant_topic,topic_name
0,55,[vapo pad]
1,87,[quit working]
2,85,[amp quot]
3,15,[keep humidifier]
4,78,[year stopped]
...,...,...
95,57,[nice product]
96,22,[clean work]
97,95,[produce steam]
98,52,[work perfect]


In [ ]:
# all_topics.to_csv('dominant_topic_0_99_2022.csv')

In [ ]:
results = df.groupby(['Dominant_topic', 'sentiment']).count().reset_index()

results['Dominant_topic'] = results['Dominant_topic'].astype(str)
all_topics['Dominant_topic'] = all_topics['Dominant_topic'].astype(str)
results = results.merge(all_topics, on='Dominant_topic')
results['topic_name'] = results['topic_name'].apply(', '.join)

graph_results = results[['topic_name', 'sentiment', 'sentiment score']]
graph_results = graph_results.pivot(index='topic_name', columns='sentiment', values='sentiment score').reset_index()

graph_results.set_index('topic_name', inplace=True)

display(graph_results)


sentiment,Negative,Neutral,Positive
topic_name,,,
accompany rating,42,21,174
amp quot,47,24,158
auto shut,54,22,176
baby room,60,28,158
best humidifier,52,29,141
...,...,...,...
work supposed,48,22,160
work wonder,44,23,152
working good,44,22,176


In [ ]:
# type(graph_results)
# graph_results.to_csv('sentiment_count_0_99_2022.csv')

**divide 2021 and 2022 data in two dataframes**

In [ ]:
# data.head()

In [ ]:
# import datetime
# data['Month_date'] = pd.to_datetime(data['Month']).dt.month
# data['year'] = pd.to_datetime(data['Month']).dt.year
# data.head()

In [ ]:
# df_2021 = data.loc[data['year']== 2021]

In [ ]:
# df_2022 = data.loc[data['year']== 2022]

In [ ]:
# df_2021.to_csv('reviews_humidifier_2021.csv',index=False)

In [ ]:
# df_2022.to_csv('reviews_humidifier_2022.csv',index=False)

In [ ]:
# df_2021.columns

In [ ]:
# df_2021.head()

In [ ]:
# df_2021['year'].value_counts()

In [ ]:
# df_2021['Review'].value_counts()
# df_2021.shape

In [ ]:
# df_2021_review = pd.DataFrame(df_2021, columns =['Review'])
# display(df_2021_review)

In [ ]:
# case text as lowercase, remove punctuation, remove extra whitespace in string and on both sides of string

# df_2021_review['remove_lower_punct'] = df_2021_review['Review'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()

# display(df_2021_review.head(10))

In [ ]:
# df_2021_review.reset_index(inplace=True)

In [ ]:
# df_2021_review

In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Aspect_Sentiment_Analysis-main/dominant_topic_0_99_2021.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Aspect_Sentiment_Analysis-main/dominant_topic_0_99_2022.csv')

In [ ]:
df1

,Unnamed: 0,Dominant_topic,topic_name
0,0,89,['humidifier hold']
1,1,97,['light bright']
2,2,76,['regular vicks']
3,3,65,['nice product']
4,4,41,['water tank']
...,...,...,...
95,95,75,['heating element']
96,96,54,['product work']
97,97,5,['menthol vapor']
98,98,16,['leave review']


In [ ]:
df2

,Unnamed: 0,Dominant_topic,topic_name
0,0,9,['season keep']
1,1,31,['good price']
2,2,79,['morning issue']
3,3,16,['humidifier time']
4,4,44,['moisture air']
...,...,...,...
95,95,76,['steam inhaler']
96,96,75,['small room']
97,97,45,['money product']
98,98,19,['winter month']


In [ ]:
# fig = graph_results.plot.bar(rot=90, figsize=(10,10))
# fig.figure.savefig('sentiment_analysis.png', bbox_inches='tight')

In [ ]:
from nltk.corpus import wordnet
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('sentiwordnet')
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        pass

positive_words = []
negative_words = []

for i in df['Dominant_topic'].unique():
    if i == 1:
        tmp_1 = df.loc[df['Dominant_topic'] == i]
                
        for j in tmp_1['tokenise'].values.tolist():
            for p in nltk.pos_tag(j):
                get_pos_tag = get_wordnet_pos(p[1])
                print(' ',get_pos_tag)
                if type(get_pos_tag) == str:
                  positive_words.append(p)
                    # try:        
                    #     synset = swn.senti_synset(p[0] + '.' + get_pos_tag +'.01')

                    #     if synset.obj_score() <= 0.30:
                    #         if synset.pos_score() > synset.neg_score() and p[0] in wn_affect:
                    #                 positive_words.append(p[0])
                    #                 # print(positive_words)
                    #         elif synset.neg_score() > synset.pos_score() and p[0] in wn_affect:
                    #                 negative_words.append(p[0])
                    #                 # print(' ',negative_words)      
                    # except:
                    #     pass
# print(' ',positive_words)
# print(' ',positive_words)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package sentiwordnet to /root/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


Streaming output truncated to the last 5000 lines.
  None
  n
  n
  r
  n
  r
  v
  None
  None
  v
  None
  a
  None
  a
  n
  None
  v
  r
  a
  None
  None
  None
  r
  v
  None
  None
  None
  a
  None
  a
  n
  r
  v
  None
  v
  None
  None
  None
  v
  n
  a
  n
  n
  a
  a
  None
  v
  v
  None
  None
  v
  r
  a
  a
  n
  None
  v
  None
  n
  n
  a
  n
  v
  r
  None
  n
  v
  v
  None
  n
  a
  n
  a
  None
  n
  v
  v
  None
  n
  None
  None
  n
  v
  v
  None
  n
  None
  a
  n
  r
  None
  None
  v
  v
  a
  a
  v
  n
  v
  None
  None
  None
  n
  r
  a
  n
  v
  n
  None
  n
  a
  r
  None
  v
  None
  None
  n
  v
  r
  v
  None
  None
  None
  None
  r
  v
  None
  v
  a
  r
  v
  v
  None
  n
  n
  None
  a
  r
  n
  v
  None
  n
  v
  None
  None
  n
  n
  a
  r
  r
  v
  r
  r
  None
  None
  None
  n
  None
  None
  n
  a
  None
  n
  v
  v
  None
  n
  None
  None
  n
  n
  v
  None
  n
  v
  n
  n
  None
  v
  r
  a
  None
  v
  r
  None
  n
  None
  v
  None
 

In [ ]:
# positive_words
negative_words

[]

In [ ]:
unique_positive_words = list(set(positive_words))
unique_negative_words = list(set(negative_words))

count_positive_words = []
count_negative_words = []

for i in unique_positive_words:
    counter = [i, positive_words.count(i)]
    count_positive_words.append(counter)

for i in unique_negative_words:
    counter = [i, negative_words.count(i)]
    count_negative_words.append(counter)    
    
positive_words = pd.DataFrame(count_positive_words, columns = ['word', 'score'])
negative_words = pd.DataFrame(count_negative_words, columns = ['word', 'score'])

positive_words.sort_values('score', ascending=False, inplace = True)
negative_words.sort_values('score', ascending=False, inplace = True)

In [ ]:
count_positive_words

[]

In [ ]:
print(positive_words)


Empty DataFrame
Columns: [word, score]
Index: []


In [ ]:
word_dict = {}
for k, v in positive_words.values:
    word_dict[k] = v

wordcloud = WordCloud()
wordcloud.generate_from_frequencies(frequencies=word_dict)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.savefig('positive_words.png')
plt.show()

ValueError: ignored

In [ ]:
# word_dict = {}
# for k, v in negative_words.values:
#     word_dict[k] = v
    
# wordcloud.generate_from_frequencies(frequencies=word_dict)
# plt.figure(figsize=(20,10))
# plt.imshow(wordcloud, interpolation="bilinear")
# plt.axis("off")
# plt.savefig('negative_words.png')
# plt.show()